In [2]:
import pandas as pd
from sqlalchemy import create_engine
from config import password

In [3]:
#Dependencies to scrap data
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import re

### Extract CSVs into DataFrames

In [9]:
car_file_path = "../top_selling_car.csv"
car_data = pd.read_csv(car_file_path)
car_brand = car_data[["#","Car brand","Car model","Release year"]]
car_brand = car_brand.rename(columns ={"#":"top","Car brand":"brand","Car model":"model","Release year":"year_release"})
car_brand["2011"]="2011"
car_brand["2012"]="2012"
car_brand["2013"]="2013"
car_brand["2014"]="2014"
car_brand["2015"]="2015"
car_brand

,top,brand,model,year_release,2011,2012,2013,2014,2015
0,1,Toyota,Corolla,1966,2011,2012,2013,2014,2015
1,2,Ford,F-Series,1977,2011,2012,2013,2014,2015
2,3,Volkswagen,Golf,1974,2011,2012,2013,2014,2015
3,4,Volkswagen,Beetle,1938,2011,2012,2013,2014,2015
4,5,Vaz,2101,1970,2011,2012,2013,2014,2015
5,6,Honda,Civic,1972,2011,2012,2013,2014,2015
6,7,Ford,Escort,1968,2011,2012,2013,2014,2015
7,8,Honda,Accord,1976,2011,2012,2013,2014,2015
8,9,Ford,Model T,1908,2011,2012,2013,2014,2015
9,10,Volkswagen,Passat,1973,2011,2012,2013,2014,2015


In [5]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
car_price=car_brand
car_price["vin_2011"]=""
car_price["vin_2012"]=""
car_price["vin_2013"]=""
car_price["vin_2014"]=""
car_price["vin_2015"]=""
car_price["2011"]=''
car_price["2012"]=''
car_price["2013"]=''
car_price["2014"]=''
car_price["2015"]=''
car_price.head()

,top,brand,model,year_release,2011,2012,2013,2014,2015,vin_2011,vin_2012,vin_2013,vin_2014,vin_2015
0,1,Toyota,Corolla,1966,,,,,,,,,,
1,2,Ford,F-Series,1977,,,,,,,,,,
2,3,Volkswagen,Golf,1974,,,,,,,,,,
3,4,Volkswagen,Beetle,1938,,,,,,,,,,
4,5,Vaz,2101,1970,,,,,,,,,,


In [8]:
#Url to get the good deal of cars
brand=car_price["brand"].astype(str)
model=car_price["model"].astype(str)
for i in range(1,6):
    year = "201"+str(i)
    year_vin_text = "vin_201"+str(i)
    for j in range(1,len(car_brand)):
        try:
            url="https://www.edmunds.com/inventory/srp.html?inventorytype=used%2Ccpo&make="+brand[j]+"&model="+model[j]+"&year="+year+"-"+year
            browser.visit(url)
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            #prices = soup.find_all('h3', class_='mb-0 text-gray-darker')
            results = soup.find_all('li', class_="search-results-inventory-card no-gutters list-unstyled bg-white mb-0_75 mb-lg-1 srp-vehicle-card hover row")
            price = results[0].find('h3',class_="mb-0 text-gray-darker").text
            href=results[0].a["href"]
            vin=re.findall('vin/(.+)/',href)
            if (vin and price):
                car_price.loc[j+1,year] = price
                car_price.loc[j+1,year_vin_text] = vin
        except (Exception, KeyError) as e:
            print(e)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
25
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
25
list index out of range
list index out of range
list index

In [16]:
car_price

,top,brand,model,year_release,2011,2012,2013,2014,2015,vin_2011,vin_2012,vin_2013,vin_2014,vin_2015
0,1.0,Toyota,Corolla,1966.0,,,,,,,,,,
1,2.0,Ford,F-Series,1977.0,,,,,,,,,,
2,3.0,Volkswagen,Golf,1974.0,,,,,,,,,,
3,4.0,Volkswagen,Beetle,1938.0,"$9,391","$9,391","$15,700",,"$16,300",WVWDM7AJ4BW213877,WVWDM7AJ0CW132117,WVWDM7AJ9DW003990,,3VW2A7AU3FM046893
4,5.0,Vaz,2101,1970.0,,"$6,991","$13,760","$16,900","$11,388",,3VWHX7AT1CM636408,3VWJP7AT3DM657215,3VWJL7AT2EM660703,3VWF17AT8FM635723
5,6.0,Honda,Civic,1972.0,,,,,,,,,,
6,7.0,Ford,Escort,1968.0,"$8,588","$9,991","$12,900","$12,995","$8,991",19XFA1F51BE045233,2HGFG4A51CH701073,19XFB2F95DE253284,19XFB2F99EE084355,19XFB2F87FE086310
7,8.0,Honda,Accord,1976.0,,,,,,,,,,
8,9.0,Ford,Model T,1908.0,"$8,490","$8,895","$8,991","$11,436","$16,691",1HGCP2F61BA107914,1HGCS1B72CA015736,1HGCR2E56DA125184,1HGCR2F37EA192360,1HGCR3F9XFA034266
9,10.0,Volkswagen,Passat,1973.0,,,,,,,,,,


In [74]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
prices = soup.find_all('h3', class_='mb-0 text-gray-darker')
results = soup.find_all('li', class_="search-results-inventory-card no-gutters list-unstyled bg-white mb-0_75 mb-lg-1 srp-vehicle-card hover row")
price = results[0].find('h3',class_="mb-0 text-gray-darker").text
a=results[0].a["href"]
vin=re.findall('vin/(.+)/',a)
print(vin)
print(price)

['19XFB2F87FE086310']
$8,991
